# Poisson: compare false positive/negatives with LASSO

LASSO is currently the de facto penalized least squares method for [feature selection](https://en.wikipedia.org/wiki/Feature_selection). Here we compare the performance (in terms of the number of false positives/negative) of LASSO with IHT for GWAS data, using the `glmnet` implementation of cyclic coordinate descent for LASSO. Since the focus here is not scalability, we test our sample problems on moderately sized genotype matrces of 1000 samples with 10,000 SNPs.

In [1]:
using Distributed
addprocs(4)
nprocs()

5

In [2]:
using MendelIHT
using SnpArrays
using DataFrames
using Distributions
using Random
using LinearAlgebra
using DelimitedFiles
using GLM
using RCall
R"library(glmnet)"

┌ Warning: RCall.jl: During startup - Warning message:
│ Setting LC_CTYPE failed, using "C" 
└ @ RCall /u/home/b/biona001/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: During startup - Warning message:
│ Setting LC_CTYPE failed, using "C" 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: During startup - Warning message:
│ Setting LC_CTYPE failed, using "C" 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: During startup - Warning message:
│ Setting LC_CTYPE failed, using "C" 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: During startup - Warning message:
│ Setting LC_CTYPE failed, using "C" 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: Matrix
│ Loading required package: foreach
│ Loaded glmnet 2.0-16
│ 
└ @ RCall /u/home/b/biona001/.julia/packages/RCall/ffM0W/src/io.jl:113


RObject{StrSxp}
 [1] "glmnet"    "foreach"   "Matrix"    "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     


In [3]:
function iht_lasso_poisson(n::Int64, p::Int64, k::Int64, d::UnionAll, l::Link)
    #construct snpmatrix, covariate files, and true model b
    x, = simulate_random_snparray(n, p, undef)
    xbm = SnpBitMatrix{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true); 
    z = ones(n, 1) # the intercept
    x_float = [convert(Matrix{Float64}, x, center=true, scale=true) z] #Float64 version of x

    # simulate response, true model b, and the correct non-0 positions of b
    y, true_b, correct_position = simulate_random_response(x, xbm, k, d, l)
 
    #specify path and folds
    num_folds = 3
    folds = rand(1:num_folds, size(x, 1));

    #run glmnet via Rcall
    @rput x_float y folds num_folds #make variables visible to R
    R"lasso_cv_result = cv.glmnet(x_float, y, nfolds = num_folds, foldid = folds, family='poisson')"
    R"lasso_beta_tmp = glmnet(x_float, y, lambda=lasso_cv_result$lambda.min, family='poisson')$beta"
    R"lasso_beta = as.vector(lasso_beta_tmp)"
    @rget lasso_cv_result lasso_beta #pull result from R to Julia
    lasso_k_est = count(!iszero, lasso_beta)
    
    #find non-zero entries returned by best lasso model as largest k estimate
    path = collect(1:50);
    
    #run IHT's cross validation routine 
    mses = cv_iht_distributed(d(), l, x, z, y, 1, path, folds, num_folds, use_maf=false, debias=false, showinfo=false, parallel=true);
    iht_k_est = argmin(mses)
    iht_result = L0_reg(x, xbm, z, y, 1, iht_k_est, d(), l, debias=false, init=false, use_maf=false)
    iht_beta = iht_result.beta
        
    #show lasso and IHT's reconstruction result
    compare_model = DataFrame(
        true_β  = true_b[correct_position], 
        IHT_β   = iht_beta[correct_position],
        lasso_β = lasso_beta[correct_position])
    @show compare_model
    
    #compute true/false positives/negatives for IHT and lasso
    iht_tp = count(!iszero, iht_beta[correct_position])
    iht_fp = iht_k_est - iht_tp
    iht_fn = k - iht_tp
    lasso_tp = count(!iszero, lasso_beta[correct_position])
    lasso_fp = lasso_k_est - lasso_tp
    lasso_fn = k - lasso_tp
    
    println("IHT false positives = $iht_fp")
    println("IHT false negatives = $iht_fn")
    println("LASSO false positives = $lasso_fp")
    println("LASSO false positives = $lasso_fn" * "\n")
    
    return iht_fp, iht_fn, lasso_fp, lasso_fn
end

iht_lasso_poisson (generic function with 1 method)

In [6]:
#simulat data with k true predictors, from distribution d and with link l.
n = 1000
p = 10000
k = 10
d = Poisson
l = canonicallink(d())

#set random seed
Random.seed!(2019)

#run function above, saving results in 4 vectors
total_runs = 50
iht_false_positives = zeros(total_runs)
iht_false_negatives = zeros(total_runs)
lasso_false_positives = zeros(total_runs)
lasso_false_negatives = zeros(total_runs)
for i in 1:total_runs
    println("current run = $i")
    iht_fp, iht_fn, lasso_fp, lasso_fn = iht_lasso_poisson(n, p, k, d, l)
    iht_false_positives[i] = iht_fp
    iht_false_negatives[i] = iht_fn
    lasso_false_positives[i] = lasso_fp
    lasso_false_negatives[i] = lasso_fn
end

current run = 1
compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β   │
│     │ Float64    │ Float64   │ Float64   │
├─────┼────────────┼───────────┼───────────┤
│ 1   │ -0.389892  │ -0.394982 │ -0.273347 │
│ 2   │ -0.0653099 │ 0.0       │ 0.0       │
│ 3   │ 0.235865   │ 0.265935  │ 0.177224  │
│ 4   │ 0.17977    │ 0.235469  │ 0.156896  │
│ 5   │ 0.0851134  │ 0.0       │ 0.0187873 │
│ 6   │ -0.33761   │ -0.306984 │ -0.18046  │
│ 7   │ 0.208012   │ 0.230687  │ 0.177488  │
│ 8   │ -0.203127  │ -0.216147 │ -0.139893 │
│ 9   │ 0.0441809  │ 0.0       │ 0.0       │
│ 10  │ 0.310431   │ 0.294478  │ 0.219388  │
IHT false positives = 0
IHT false negatives = 3
LASSO false positives = 42
LASSO false positives = 2

current run = 2
compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β    │
│     │ Float64    │ Float64   │ Float64    │
├─────┼────────────┼───────────┼────────────┤
│ 1   │ -0.261627  │ -0.324249 │ -0.246127  │
│ 2   │ -0.153536  │ -0.170536 │ -0

┌ Warning: RCall.jl: Warning: from glmnet Fortran code (error code -1); Convergence for 1th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned
│ Warning in getcoef(fit, nvars, nx, vnames) :
│   an empty model has been returned; probably a convergence issue
└ @ RCall /u/home/b/biona001/.julia/packages/RCall/ffM0W/src/io.jl:113


compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β │
│     │ Float64    │ Float64   │ Float64 │
├─────┼────────────┼───────────┼─────────┤
│ 1   │ 0.434107   │ 0.454195  │ 0.0     │
│ 2   │ -0.157956  │ 0.0       │ 0.0     │
│ 3   │ 0.00266164 │ 0.0       │ 0.0     │
│ 4   │ -0.107333  │ 0.0       │ 0.0     │
│ 5   │ -0.25618   │ -0.231934 │ 0.0     │
│ 6   │ 0.663682   │ 0.71418   │ 0.0     │
│ 7   │ 0.375309   │ 0.377542  │ 0.0     │
│ 8   │ -0.107927  │ 0.0       │ 0.0     │
│ 9   │ -0.476671  │ -0.479865 │ 0.0     │
│ 10  │ -0.0818371 │ 0.0       │ 0.0     │
IHT false positives = 0
IHT false negatives = 5
LASSO false positives = 0
LASSO false positives = 10

current run = 15
compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β    │
│     │ Float64    │ Float64   │ Float64    │
├─────┼────────────┼───────────┼────────────┤
│ 1   │ 0.0363198  │ 0.0       │ 0.0        │
│ 2   │ -0.501696  │ -0.521192 │ -0.426966  │
│ 3   │ -0.153984  │ 0.0      

┌ Warning: RCall.jl: Warning: from glmnet Fortran code (error code -1); Convergence for 1th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned
│ Warning in getcoef(fit, nvars, nx, vnames) :
│   an empty model has been returned; probably a convergence issue
└ @ RCall /u/home/b/biona001/.julia/packages/RCall/ffM0W/src/io.jl:113


Did not converge!!!!! The run time for IHT was 14.45557188987732 seconds and model size was11
compare_model = 10×3 DataFrame
│ Row │ true_β    │ IHT_β   │ lasso_β │
│     │ Float64   │ Float64 │ Float64 │
├─────┼───────────┼─────────┼─────────┤
│ 1   │ 0.325646  │ 0.0     │ 0.0     │
│ 2   │ 0.170934  │ 0.0     │ 0.0     │
│ 3   │ -0.219413 │ 0.0     │ 0.0     │
│ 4   │ 0.0261994 │ 0.0     │ 0.0     │
│ 5   │ 0.688261  │ 58.352  │ 0.0     │
│ 6   │ 0.120471  │ 0.0     │ 0.0     │
│ 7   │ -0.263155 │ 0.0     │ 0.0     │
│ 8   │ 0.109031  │ 0.0     │ 0.0     │
│ 9   │ 0.238471  │ 0.0     │ 0.0     │
│ 10  │ -0.551881 │ 0.0     │ 0.0     │
IHT false positives = 10
IHT false negatives = 9
LASSO false positives = 0
LASSO false positives = 10

current run = 18
compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β    │
│     │ Float64    │ Float64   │ Float64    │
├─────┼────────────┼───────────┼────────────┤
│ 1   │ -0.277114  │ -0.241048 │ -0.15846   │
│ 2   │ 0.178041   │

...

compare_model = 10×3 DataFrame
│ Row │ true_β    │ IHT_β     │ lasso_β │
│     │ Float64   │ Float64   │ Float64 │
├─────┼───────────┼───────────┼─────────┤
│ 1   │ 0.28703   │ 0.284733  │ 0.0     │
│ 2   │ -0.483911 │ -0.478402 │ 0.0     │
│ 3   │ -0.287957 │ -0.270259 │ 0.0     │
│ 4   │ 0.319652  │ 0.262953  │ 0.0     │
│ 5   │ 0.173612  │ 0.12942   │ 0.0     │
│ 6   │ 0.10954   │ 0.111648  │ 0.0     │
│ 7   │ 0.754705  │ 0.75976   │ 0.0     │
│ 8   │ -0.444441 │ -0.462245 │ 0.0     │
│ 9   │ 0.0930373 │ 0.118968  │ 0.0     │
│ 10  │ -0.608667 │ -0.591464 │ 0.0     │
IHT false positives = 1
IHT false negatives = 0
LASSO false positives = 0
LASSO false positives = 10

current run = 35
compare_model = 10×3 DataFrame
│ Row │ true_β    │ IHT_β    │ lasso_β    │
│     │ Float64   │ Float64  │ Float64    │
├─────┼───────────┼──────────┼────────────┤
│ 1   │ 0.300981  │ 0.337873 │ 0.302699   │
│ 2   │ 0.221178  │ 0.234555 │ 0.169457   │
│ 3   │ -0.112423 │ -0.13765 │ -0.0535475 │
│ 4   │ 0

┌ Warning: RCall.jl: Warning: from glmnet Fortran code (error code -1); Convergence for 1th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned
│ Warning in getcoef(fit, nvars, nx, vnames) :
│   an empty model has been returned; probably a convergence issue
└ @ RCall /u/home/b/biona001/.julia/packages/RCall/ffM0W/src/io.jl:113


compare_model = 10×3 DataFrame
│ Row │ true_β    │ IHT_β     │ lasso_β │
│     │ Float64   │ Float64   │ Float64 │
├─────┼───────────┼───────────┼─────────┤
│ 1   │ 0.282284  │ 0.308155  │ 0.0     │
│ 2   │ 0.630258  │ 0.620913  │ 0.0     │
│ 3   │ 0.198703  │ 0.228435  │ 0.0     │
│ 4   │ -0.241361 │ -0.243096 │ 0.0     │
│ 5   │ -0.203833 │ -0.206239 │ 0.0     │
│ 6   │ 0.450464  │ 0.454754  │ 0.0     │
│ 7   │ 0.0999571 │ 0.0       │ 0.0     │
│ 8   │ 0.110379  │ 0.0       │ 0.0     │
│ 9   │ -0.121467 │ 0.0       │ 0.0     │
│ 10  │ -0.344978 │ -0.373391 │ 0.0     │
IHT false positives = 1
IHT false negatives = 3
LASSO false positives = 0
LASSO false positives = 10

current run = 38
compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β   │
│     │ Float64    │ Float64   │ Float64   │
├─────┼────────────┼───────────┼───────────┤
│ 1   │ 0.0252926  │ 0.0       │ 0.0       │
│ 2   │ -0.293336  │ -0.249928 │ -0.105991 │
│ 3   │ -0.230579  │ -0.277923 │ -0.184645 │
│ 4

┌ Warning: RCall.jl: Warning: from glmnet Fortran code (error code -1); Convergence for 1th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned
│ Warning in getcoef(fit, nvars, nx, vnames) :
│   an empty model has been returned; probably a convergence issue
└ @ RCall /u/home/b/biona001/.julia/packages/RCall/ffM0W/src/io.jl:113


compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β │
│     │ Float64    │ Float64   │ Float64 │
├─────┼────────────┼───────────┼─────────┤
│ 1   │ 0.295306   │ 0.322279  │ 0.0     │
│ 2   │ -0.0336288 │ 0.0       │ 0.0     │
│ 3   │ 0.28116    │ 0.284369  │ 0.0     │
│ 4   │ 0.569635   │ 0.564957  │ 0.0     │
│ 5   │ -0.0437276 │ 0.0       │ 0.0     │
│ 6   │ 0.32834    │ 0.326908  │ 0.0     │
│ 7   │ -0.319752  │ -0.30664  │ 0.0     │
│ 8   │ 0.609374   │ 0.604262  │ 0.0     │
│ 9   │ 0.475774   │ 0.448355  │ 0.0     │
│ 10  │ 0.0905152  │ 0.0994956 │ 0.0     │
IHT false positives = 0
IHT false negatives = 2
LASSO false positives = 0
LASSO false positives = 10

current run = 41
compare_model = 10×3 DataFrame
│ Row │ true_β     │ IHT_β     │ lasso_β   │
│     │ Float64    │ Float64   │ Float64   │
├─────┼────────────┼───────────┼───────────┤
│ 1   │ 0.118173   │ 0.0       │ 0.0391657 │
│ 2   │ 0.141545   │ 0.140278  │ 0.0783399 │
│ 3   │ 0.135242   │ 0.0       │ 0.

In [7]:
poisson_iht_false_positives = sum(iht_false_positives) / 50
poisson_iht_false_negatives = sum(iht_false_negatives) / 50
poisson_lasso_false_positives = sum(lasso_false_positives) / 50
poisson_lasso_false_negatives = sum(lasso_false_negatives) / 50
result = [poisson_iht_false_positives; poisson_iht_false_negatives; 
         poisson_lasso_false_positives; poisson_lasso_false_negatives]

4-element Array{Float64,1}:
  0.5 
  3.32
 29.74
  3.52

In [1]:
IHT_did_not_converge = 0
LASSO_did_not_converge = 7

7